In [9]:
# This file was based on
# https://github.com/openai/baselines/blob/edb52c22a5e14324304a491edc0f91b6cc07453b/baselines/common/atari_wrappers.py
# its license:
#
# The MIT License
#
# Copyright (c) 2017 OpenAI (http://openai.com)
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

from collections import deque

import cv2
import gym
import numpy as np
from gym import spaces

cv2.ocl.setUseOpenCL(False)


class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.integers(1, self.noop_max + 1)  # pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)


class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)


class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs


class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,) + env.observation_space.shape, dtype=np.uint8)
        self._skip = skip

    def reset(self):
        return self.env.reset()

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255,
                                            shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.

        Returns lazy array, which is much more memory efficient.

        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0], shp[1], shp[2] * k), dtype=np.uint8)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))


class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0


class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.

        This object should only be converted to numpy array before being passed to the model.

        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=2)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]


def make_atari(env_id, render_mode=None,max_episode_steps=400000):
    if render_mode == None:
        env = gym.make(env_id)
    elif render_mode == 'human':
        env = gym.make(env_id,render_mode='human')
    env._max_episode_steps = max_episode_steps
#     assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=1)
    env = MaxAndSkipEnv(env, skip=4)
    return env


def wrap_deepmind(env, episode_life=False, clip_rewards=False, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari."""
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env


In [467]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class DQN(nn.Module):

    def __init__(self, h, w, outputs, device):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4, bias=False)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, bias=False)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, bias=False)
        self.fc1 = nn.Linear(64*7*7, 512)
        self.fc2 = nn.Linear(512, outputs)
        self.device = device
    
    def init_weights(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            m.bias.data.fill_(0.0)
        
        if type(m) == nn.Conv2d:
            torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            #m.bias.data.fill_(0.1)
    
    def forward(self, x):
        x = x.to(self.device).float() / 255.
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc1(x.view(x.size(0), -1)))
        return self.fc2(x)

class ActionSelector(object):
    def __init__(self, INITIAL_EPSILON, FINAL_EPSILON, policy_net, EPS_DECAY, n_actions, device):
        self._eps = INITIAL_EPSILON
        self._FINAL_EPSILON = FINAL_EPSILON
        self._INITIAL_EPSILON = INITIAL_EPSILON
        self._policy_net = policy_net
        self._EPS_DECAY = EPS_DECAY
        self._n_actions = n_actions
        self._device = device

    def select_action(self, state, training=False):
        sample = random.random()
        if training:
            self._eps -= (self._INITIAL_EPSILON - self._FINAL_EPSILON)/self._EPS_DECAY
            self._eps = max(self._eps, self._FINAL_EPSILON)
        if sample > self._eps:
            with torch.no_grad():
            	a = self._policy_net(state).max(1)[1].cpu().view(1,1)
        else:
            a = torch.tensor([[random.randrange(self._n_actions)]], device='cpu', dtype=torch.long)
        return a.numpy()[0,0].item(), self._eps

class ReplayMemory(object):
    def __init__(self, capacity, state_shape, n_actions, device):
        c,h,w = state_shape
        self.capacity = capacity
        self.device = device
        self.m_states = torch.zeros((capacity, c, h, w), dtype=torch.uint8)
        self.m_actions = torch.zeros((capacity, 1), dtype=torch.long)
        self.m_rewards = torch.zeros((capacity, 1), dtype=torch.int8)
        self.m_dones = torch.zeros((capacity, 1), dtype=torch.bool)
        self.position = 0
        self.size = 0

    def push(self, state, action, reward, done):
        """Saves a transition."""
        self.m_states[self.position] = state # 5,84,84
        self.m_actions[self.position,0] = action
        self.m_rewards[self.position,0] = reward
        self.m_dones[self.position,0] = done
        self.position = (self.position + 1) % self.capacity
        self.size = max(self.size, self.position)

    def sample(self, bs):
        i = torch.randint(0, high=self.size, size=(bs,))
        bs = self.m_states[i, :4] 
        bns = self.m_states[i, 1:] 
        ba = self.m_actions[i].to(self.device)
        br = self.m_rewards[i].to(self.device).float()
        bd = self.m_dones[i].to(self.device).float()
        return bs, ba, br, bns, bd

    def __len__(self):
        return self.size

def fp(n_frame):
    n_frame = torch.from_numpy(n_frame)
    h = n_frame.shape[-2]
    return n_frame.view(1,h,h)

class FrameProcessor():
    def __init__(self, im_size=84):
        self.im_size = im_size

    def process(self, frame):
        im_size = self.im_size
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = frame[46:160+46, :]

        frame = cv2.resize(frame, (im_size, im_size), interpolation=cv2.INTER_LINEAR)
        frame = frame.reshape((1, im_size, im_size))

        x = torch.from_numpy(frame)
        return x




class PrioritizedReplayBuffer(object):
    def __init__(self, capacity, state_shape, n_actions, device):
        c,h,w = state_shape
        self.capacity = capacity
        self.device = device
        self.m_states = torch.zeros((capacity, c, h, w), dtype=torch.uint8)
        self.m_actions = torch.zeros((capacity, 1), dtype=torch.long)
        self.m_rewards = torch.zeros((capacity, 1), dtype=torch.int8)
        self.m_dones = torch.zeros((capacity, 1), dtype=torch.bool)
        self.priorities = np.zeros(capacity)
        self.position = 0
        self.size = 0

    def push(self, state, action, reward, done, priority):
        """Saves a transition."""
        self.m_states[self.position] = state # 5,84,84
        self.m_actions[self.position,0] = action
        self.m_rewards[self.position,0] = reward
        self.m_dones[self.position,0] = done
        self.priorities[self.position] = priority
        self.position = (self.position + 1) % self.capacity
        self.size = max(self.size, self.position)

    def getPriorities(self,bs):

        arr = self.priorities

        # Create a boolean array indicating which elements are not zero
        non_zero_mask = arr != 0

        # Use the boolean array to get a subset of the original array
        
        tempPrio = arr[non_zero_mask]
        prioritySum = np.sum(tempPrio)
        sampling_probabilities = tempPrio / prioritySum
        indexValues = []

        #Sample random integers given probability
        for i in range(bs):
            indexValues.append(np.random.choice(len(sampling_probabilities), p=sampling_probabilities))
        return indexValues

    def sample(self, bs):
        i = self.getPriorities(bs)
        bs = self.m_states[i, :4] 
        bns = self.m_states[i, 1:] 
        ba = self.m_actions[i].to(self.device)
        br = self.m_rewards[i].to(self.device).float()
        bd = self.m_dones[i].to(self.device).float()
        return bs, ba, br, bns, bd

    def foo(self,bs):
        i = torch.randint(0, high=self.size, size=(bs,))
        return i
    def __len__(self):
        return self.size

In [468]:
print(len(memory.m_states))
print(len(memory.priorities))

10000
10000


In [469]:
import math
import random
import numpy as np
import os
from collections import deque
from datetime import datetime
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [470]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if gpu is to be used

# 3. environment reset
env_name = 'Breakout-v4'
#env_name = 'SpaceInvaders'
#env_name = 'Riverraid'
#env_name = 'Seaquest
#env_name = 'MontezumaRevenge'
env_raw = make_atari(env_name)
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=True, clip_rewards=True)

c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n
print(n_actions)


4


In [471]:
# 4. Network reset
policy_net = DQN(h, w, n_actions, device).to(device)
target_net = DQN(h, w, n_actions, device).to(device)
policy_net.apply(policy_net.init_weights)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()


DQN(
  (conv1): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4), bias=False)
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), bias=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (fc1): Linear(in_features=3136, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=4, bias=True)
)

In [472]:

priorities = torch.zeros((5, 1), dtype=torch.float32)

print(priorities)

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]])


In [473]:
# # 5. DQN hyperparameters
# BATCH_SIZE = 32
# GAMMA = 0.99
# EPS_START = 1.
# EPS_END = 0.1
# TARGET_UPDATE = 5000
# # NUM_EPS = 1000
# NUM_STEPS = 500000
# M_SIZE = 10000
# POLICY_UPDATE = 4
# EVALUATE_FREQ = 2000

# EPS_DECAY = (EPS_START-EPS_END) / (EPS_END / NUM_STEPS)

optimizer = optim.Adam(policy_net.parameters(), lr=0.0000625, eps=1.5e-4)



BATCH_SIZE = 32
GAMMA = 0.99
EPS_START = 1.
EPS_END = 0.1
EPS_DECAY = 440000
#how many steps until target updates
TARGET_UPDATE = 5000
NUM_STEPS = 500000
M_SIZE = 10000
POLICY_UPDATE = 4
EVALUATE_FREQ = 2000

In [474]:
# BATCH_SIZE = 32
# GAMMA = 0.99
# EPS_START = 1.
# EPS_END = 0.1
# TARGET_UPDATE = 5000
# # NUM_EPS = 1000
# NUM_STEPS = 500000
# M_SIZE = 10000
# POLICY_UPDATE = 4
# EVALUATE_FREQ = 2000

# EPS_DECAY = (EPS_START-EPS_END) / (EPS_END / NUM_STEPS)

optimizer = optim.Adam(policy_net.parameters(), lr=0.0000625, eps=1.5e-4)



BATCH_SIZE = 32
GAMMA = 0.99
EPS_START = 1.
EPS_END = 0.1
EPS_DECAY = 440000
#how many steps until target updates
TARGET_UPDATE = 5000
NUM_STEPS = 500000
M_SIZE = 10000
POLICY_UPDATE = 4
EVALUATE_FREQ = 2000

In [475]:
# replay memory and action selector
memory = PrioritizedReplayBuffer(M_SIZE, [5,h,w], n_actions, device)
sa = ActionSelector(EPS_START, EPS_END, policy_net, EPS_DECAY, n_actions, device)


In [476]:
steps_done = 0
q = deque(maxlen=5)
done = True
eps = 0
episode_len = 0
totalEpisodes = 0

In [477]:
def getPriority(state,reward,done):
    tState = torch.tensor(state)

    qV = policy_net(tState).max(1)[0].detach()
    nq = target_net(tState).max(1)[0].detach()
    
    doneBatch = torch.tensor(np.array([int(done)]))

    expected_state_action_values = (nq * GAMMA)*(1.-doneBatch) + reward

    loss = F.smooth_l1_loss(qV, expected_state_action_values)

    return abs(loss)


def optimize_model(train):
    if not train:
        return
    state_batch, action_batch, reward_batch, n_state_batch, done_batch = memory.sample(BATCH_SIZE)

    q = policy_net(state_batch).gather(1, action_batch)
    nq = target_net(n_state_batch).max(1)[0].detach()

    # Compute the expected Q values
    expected_state_action_values = (nq * GAMMA)*(1.-done_batch[:,0]) + reward_batch[:,0]

    # Compute Huber loss
    loss = F.smooth_l1_loss(q, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

    

In [478]:
def evaluate(step, policy_net, device, env, n_actions, curr_eps,total_eps,train ,eps=0.05 ,num_episode=5):
    env = wrap_deepmind(env)
    sa = ActionSelector(eps, eps, policy_net, EPS_DECAY, n_actions, device)
    e_rewards = []
    q = deque(maxlen=5)
    for i in range(num_episode):
        env.reset()
        e_reward = 0
        for _ in range(10): # no-op
            n_frame, _, done, _ = env.step(0)
            n_frame = fp(n_frame)
            q.append(n_frame)

        while not done:
            state = torch.cat(list(q))[1:].unsqueeze(0)
            action, eps = sa.select_action(state, train)
            n_frame, reward, done, info = env.step(action)
            n_frame = fp(n_frame)
            q.append(n_frame)
            
            e_reward += reward
        e_rewards.append(e_reward)

    f = open("file.txt",'a') 
    f.write("Average reward: %f, Steps: %d, number of eps: %d, current eps: %f, total eps: %f\n" % (float(sum(e_rewards))/float(num_episode), step, num_episode,float(curr_eps),float(total_eps)))
    f.close()

In [479]:
steps_done = 0
q = deque(maxlen=5)
done = True
eps = 0
episode_len = 0
totalEpisodes = 0

In [480]:
# progressive = tqdm(range(NUM_STEPS), total=NUM_STEPS, ncols=50, leave=False, unit='b')
# for step in progressive:
#     if done: # life reset !!!
#         totalEpisodes +=1
#         env.reset()
#         sum_reward = 0
#         episode_len = 0
#         img, _, _, _ = env.step(1) # BREAKOUT specific !!!
#         for i in range(10): # no-op
#             n_frame, _, _, _ = env.step(0)
#             n_frame = fp(n_frame)
#             q.append(n_frame)
    
#     train = len(memory) > EVALUATE_FREQ
#     # Select and perform an action
#     state = torch.cat(list(q))[1:].unsqueeze(0)
#     action, cuur_eps = sa.select_action(state, train)
#     n_frame, reward, done, info = env.step(action)
#     n_frame = fp(n_frame)

#     # 5 frame as memory
#     q.append(n_frame)
#     memory.push(torch.cat(list(q)).unsqueeze(0), action, reward, done) # here the n_frame means next frame from the previous time step
#     episode_len += 1

#     # Perform one step of the optimization (on the target network)
#     if step % POLICY_UPDATE == 0:
#         optimize_model(train)
    
#     # Update the target network, copying all weights and biases in DQN
#     if step % TARGET_UPDATE == 0:
#         target_net.load_state_dict(policy_net.state_dict())
    
#     if step % EVALUATE_FREQ == 0:
#         evaluate(step, policy_net, device, env_raw, n_actions,cuur_eps,totalEpisodes,train ,eps=0.05, num_episode=15)

In [481]:
progressive = tqdm(range(NUM_STEPS), total=NUM_STEPS, ncols=50, leave=False, unit='b')
for step in progressive:
    if done: # life reset !!!
        totalEpisodes +=1
        env.reset()
        sum_reward = 0
        episode_len = 0
        img, _, _, _ = env.step(1) # BREAKOUT specific !!!
        for i in range(10): # no-op
            n_frame, _, _, _ = env.step(0)
            n_frame = fp(n_frame)
            q.append(n_frame)
    
    train = len(memory) > EVALUATE_FREQ
    # Select and perform an action
    state = torch.cat(list(q))[1:].unsqueeze(0)
    action, cuur_eps = sa.select_action(state, train)
    n_frame, reward, done, info = env.step(action)
    n_frame = fp(n_frame)

    #Get priority
    priority = getPriority(state,reward,done)

    # 5 frame as memory
    q.append(n_frame)
    # memory.push(torch.cat(list(q)).unsqueeze(0), action, reward, done) # here the n_frame means next frame from the previous time step
    memory.push(torch.cat(list(q)).unsqueeze(0), action, reward, done,priority) # here the n_frame means next frame from the previous time step
    episode_len += 1

    # Perform one step of the optimization (on the target network)
    if step % POLICY_UPDATE == 0:
        optimize_model(train)
    
    # Update the target network, copying all weights and biases in DQN
    if step % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
    
    if step % EVALUATE_FREQ == 0:
        evaluate(step, policy_net, device, env_raw, n_actions,cuur_eps,totalEpisodes,train ,eps=0.05, num_episode=15)

  0%|                   | 0/500000 [00:00<?, ?b/s]/var/folders/2z/0p788lv54yddrxzp4bhbsyj80000gn/T/ipykernel_25318/3067137549.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tState = torch.tensor(state)


KeyboardInterrupt: 